<a href="https://colab.research.google.com/github/Kathirvel200508/RAG_pipeline/blob/main/RAG_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langchain-community faiss-cpu sentence-transformers openai pypdf
!pip install -q langchain-community langchain-text-splitters faiss-cpu sentence-transformers openai pypdf



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"


In [ ]:
from google.colab import files

uploaded = files.upload()
txt_name = list(uploaded.keys())[0]
print("Uploaded:", txt_name)


Saving INDIAN_CONSTITUTION.txt to INDIAN_CONSTITUTION.txt
Uploaded: INDIAN_CONSTITUTION.txt


In [ ]:
from langchain_core.documents import Document

with open(txt_name, "r", encoding="utf-8") as f:
    text = f.read()

docs = [Document(page_content=text)]

print("Documents:", len(docs))
print(docs[0].page_content[:300])




Documents: 1
£ÉÉ®iÉ BÉEÉ ºÉÆÉÊ´ÉvÉÉxÉ 
[1
, 2024
]
THE CONSTITUTION OF INDIA
[As on 1st May, 2024] 
2024
GOVERNMENT OF INDIA
MINISTRY OF LAW AND JUSTICE
LEGISLATIVE DEPARTMENT, OFFICIAL LANGUAGES WING


PREFACE
This is the  sixth pocket size edition of the Constitution of 
India in the diglot form. In this editi


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=240
)

documents = text_splitter.split_documents(docs)
print("Chunks:", len(documents))


Chunks: 1224


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)


### Save the FAISS vector store

In [ ]:
vectorstore.save_local("faiss_index_supreme_court")

### Load the FAISS vector store (in a new session or if you restart the runtime)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Re-initialize embeddings, as it's needed to load the vectorstore
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.load_local("faiss_index_supreme_court", embeddings, allow_dangerous_deserialization=True)

In [ ]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name="meta-llama/llama-3-8b-instruct",
    temperature=0
)


/tmp/ipython-input-2272593246.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
def rag_answer(question, k=8):
    # 1. Retrieve relevant chunks
    docs = vectorstore.similarity_search(question, k=k)

    context = "\n\n".join(
        [f"Source {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)]
    )

    # 2. Build prompt
    prompt = f"""
You are answering strictly and ONLY from the provided context.
Do NOT use outside knowledge.
Do NOT answer if the information is not present in the context.
If the answer is not found in the context, reply exactly with:
"Not available in the provided document."

Context:
{context}

Question:
{question}

Answer:
"""

    # 3. Call LLM
    return llm.invoke(prompt).content



In [ ]:
question = "What does the Preamble of the Constitution of India state?"
answer = rag_answer(question)

print(answer)


WE, THE PEOPLE OF INDIA, having solemnly resolved to constitute 
India into a 
1[SOVEREIGN SOCIALIST SECULAR DEMOCRATIC 
REPUBLIC] and to secure to all its citizens:
JUSTICE, social, economic and political;
 
LIBERTY of thought, expression, belief, faith and worship;
EQUALITY of status and of opportunity;
and to promote among them all
FRATERNITY assuring the dignity of the individual and the 2[unity 
and integrity of the Nation];
